In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pyiast
import scipy.optimize as optim
import os
import matplotlib.pyplot as plt
import pickle
import csv
from scipy.optimize import curve_fit

In [2]:
def langmuir(p,M,K):
    return M*p*K/(1+p*K)

In [3]:
data_all = pd.read_excel("IVSAresults.xlsx")
M_list_all = data_all[["Xe_M_pre","Kr_M_pre"]]
h_list_all = data_all[["Xe_henry","Kr_henry"]]
Name_list_all = data_all["Filename"]
H_list_all = data_all[["Xe_heat","Kr_heat"]]

In [4]:
Arrh = lambda T,dH ,T_ref: np.exp(-dH/8.3145*(1/T - 1/T_ref)) # Arrhenius equation (Clasius-Clapeyron Equation)

In [5]:
def iso_mix(P_par, T, M_list, h_list, dH_list):
    
    p= np.linspace(0,100,100)
    i=0
    q_Xe=np.zeros(100)
    q_Kr=np.zeros(100)
    
    for i in range(0,100):
        Xe_M = M_list[0]
        Kr_M = M_list[1]
        Xe_K = h_list[0]*100000/Xe_M
        Kr_K = h_list[1]*100000/Kr_M
        q_Xe[i] = langmuir(p[i],Xe_M,Xe_K)
        q_Kr[i] = langmuir(p[i],Kr_M,Kr_K)
        i = i+1
        
    iso_Xe={"p" : p, "q" : q_Xe}
    iso_Kr={"p" : p,"q" : q_Kr}
    data_Xe=pd.DataFrame(iso_Xe)
    data_Kr=pd.DataFrame(iso_Kr)
    
    df_ISO_list =[data_Xe,data_Kr]
    iso_list = [pyiast.ModelIsotherm(df_ISO_list[0],loading_key = 'q', pressure_key = 'p',model = 'Langmuir',param_guess={"M": Xe_M,"K":Xe_K}),
               pyiast.ModelIsotherm(df_ISO_list[1],loading_key = 'q', pressure_key = 'p',model = 'Langmuir',param_guess={"M": Kr_M,"K":Kr_K})]

    P_norm = []
    for (p,dh,tref) in zip(P_par, dH_list,[T, T]):
        p_n = Arrh(T,dh,tref)*p 
        P_norm.append(p_n)
    P_norm_arr = np.array(P_norm)

    if P_norm_arr.ndim > 1:
        for i in range(len(P_norm[0])):
            p_tmp = P_norm_arr[i,:]
            p_tmp[p_tmp<0.000001] = 0.000001
            q_IAST_tmp = pyiast.iast(p_tmp,
                                     iso_list,
                                     warningoff=True)
    else:
        try:
            p_tmp = P_norm_arr
            p_tmp[p_tmp<0.000001] = 0.000001
          
            q_IAST_tmp = pyiast.iast(p_tmp,
                                    iso_list,
                                     warningoff=True)
        except:    
            try:
                x_IG = np.ones(len(p_tmp))/len(p_tmp)
                q_IAST_tmp = pyiast.iast(p_tmp,
                                        iso_list,adsorbed_mole_fraction_guess = x_IG,
                                        warningoff=True)
            except:
                try:
                    arg_min = np.argmin(p_tmp)
                    p_tmp[p_tmp<0.000001] = 0.000001
                    x_IG = 0.05*np.ones(len(p_tmp))
                    x_IG[arg_min] = 1 - 0.05*(len(p_tmp)-1)
                    q_IAST_tmp = pyiast.iast(p_tmp,
                                            iso_list,adsorbed_mole_fraction_guess = x_IG,
                                            warningoff=True)

                except:
                    try:
                        arg_max = np.argmax(p_tmp)
                        p_tmp[p_tmp<0.000001] = 0.000001
                        x_IG = 0.05*np.ones(len(p_tmp))
                        x_IG[arg_max] = 1 - 0.05*(len(p_tmp)-1)
        
                        q_IAST_tmp = pyiast.iast(p_tmp,
                                                iso_list,adsorbed_mole_fraction_guess = x_IG,
                                                warningoff=True)        
                    except:
                        try:
                            arg_max = np.argmax(p_tmp)
                            p_tmp[p_tmp<0.000001] = 0.000001
                            x_IG = 0.15*np.ones(len(p_tmp))
                            x_IG[arg_max] = 1 - 0.15*(len(p_tmp)-1)
                            q_IAST_tmp = pyiast.iast(p_tmp,
                                                iso_list,adsorbed_mole_fraction_guess = x_IG,
                                                warningoff=True)
                        except:
                            try:
                                arg_min = np.argmin(p_tmp)
                                p_tmp[p_tmp<0.000001] = 0.000001
                                x_IG = 0.01*np.ones(len(p_tmp))
                                x_IG[arg_min] = 1 - 0.01*(len(p_tmp)-1)
                      
                                q_IAST_tmp = pyiast.iast(p_tmp,
                                            iso_list,adsorbed_mole_fraction_guess = x_IG,
                                            warningoff=True)

                            except:
                                arg_max = np.argmax(p_tmp)
                                p_tmp[p_tmp<0.000001] = 0.000001
                                x_IG = 0.01*np.ones(len(p_tmp))
                                x_IG[arg_max] = 1 - 0.01*(len(p_tmp)-1)
                    
                                q_IAST_tmp = pyiast.iast(p_tmp,
                                                iso_list,adsorbed_mole_fraction_guess = x_IG,
                                            warningoff=True)
    
    S = q_IAST_tmp[0]*0.08/q_IAST_tmp[1]/0.02
           
    return q_IAST_tmp,S

In [6]:
qH = []
qL = []

In [17]:
for M_list,h_list,dH,nam in zip(M_list_all.values,h_list_all.values,H_list_all.values,Name_list_all):
    q1,S1,= iso_mix([0.02,0.08], 298, M_list, h_list, dH)
    qL.append(q1)
    print(nam,q1,S1)

ABAYIO_clean [0.10019643 0.05324567] 7.527105274509872
ABAYOU_clean [0.10440966 0.05577373] 7.488089355442994
ABEMIF_clean [0.1004433  0.06036159] 6.656107172811865
ABESUX_clean [0.01969573 0.01419269] 5.550951930838372
ABETAE_clean [0.01920886 0.01404026] 5.4725099572884615
ABETIN_clean [0.03009156 0.03754629] 3.2058089905623186
ABEXEM_clean [0.03418309 0.02156537] 6.340365611324196
ABEXEN_clean [0.05702497 0.04763835] 4.788157244315819
ABEXIQ_clean [0.04447601 0.0254082 ] 7.001835803580674
ABEXOW_clean [0.03344981 0.02076073] 6.444824652928695
ABEXUC_clean [0.03357427 0.02086399] 6.436787086353522
ABEYAJ_clean [0.03382296 0.02025131] 6.680645755913426
ABEYEN_clean [0.03296238 0.01966113] 6.706098653690367
ABEYIR_clean [0.03252462 0.01956061] 6.651041579836334
ABIXOZ_clean [0.00816746 0.00932373] 3.503944682913791
ABULOB_clean [0.53253159 0.17274385] 12.33112715260709
ABUWOJ_clean [0.07658352 0.05545674] 5.523838418544146
ac403674p_si_001_clean [0.01419553 0.01433402] 3.96135324419334

AMOYOR_clean [0.17841091 0.06621223] 10.778124454579109
AMUFOF_clean [0.05644028 0.03114328] 7.249113160366023
AMUWIP_charged [0.0332269  0.03501683] 3.7955355439607965
AMUWUB_clean [0.01768703 0.01864502] 3.79447844785116
AMUXAI_clean [0.01747564 0.01695779] 4.122149962237703
ANAWIX_clean [0.10338448 0.05026542] 8.227084922599637
ANAWOD_clean [0.08581722 0.05185266] 6.620081671633156
ANENEN_clean [0.06558113 0.03425196] 7.658672152780895
ANENIR_clean [0.07674457 0.03671497] 8.361119042045912
ANIDIL_clean [0.03476416 0.01909699] 7.281601367744209
ANOMUM_clean [0.07647753 0.06041491] 5.063486635295393
ANONAT_manual [0.06318777 0.0535771 ] 4.717521162322696
ANUGEW_clean [0.01871776 0.03004342] 2.49209534436536
ANUGIA_clean [0.02692491 0.0355085 ] 3.033066278725418
ANUGOG_clean [0.02990589 0.03805047] 3.1438129148596774
ANUGUM_clean [0.06210431 0.05928772] 4.190028729844293
APACAX_clean [0.17457682 0.09614948] 7.2627256447464985
APARAM_clean [0.3027526  0.07746685] 15.632627159871001
APAY

AZAVUU_clean [0.29652649 0.10683981] 11.101722945550197
AZAWAB_clean [0.19356509 0.08401362] 9.21589060386168
AZEBUE_clean [0.24251052 0.08168592] 11.875266135520942
AZIVAI_clean [0.06148849 0.05601945] 4.390509892931542
AZIXUD_clean [0.09438076 0.05876413] 6.424378282700096
BACGAQ_clean [0.12162924 0.05796671] 8.393040591556494
BADXUC_clean [0.01900967 0.02183967] 3.481677299185939
BADYAJ_clean [0.02565907 0.02626008] 3.908453557286134
BADZAK_clean [0.0459392  0.03968754] 4.630087873546574
BAFHAU_clean [0.00595279 0.01078808] 2.207173213117468
BAKGIF_manual [0.03920047 0.03278703] 4.782436459397131
BAKJEE_clean [0.10109023 0.06702545] 6.032945583265884
BAKJII_clean [0.12010166 0.07722125] 6.221171693614703
BAKJOO_clean [0.07161357 0.05168969] 5.54180733078419
BAKYOE_clean [0.12113061 0.07049097] 6.873538971767692
BAMKIM_clean [0.02496638 0.02211622] 4.515486412453916
BAMKOS_clean [0.02752993 0.02392982] 4.601777061328482
BAMMOU_clean [0.01349827 0.0142769 ] 3.7818491265756253
BAMMUA_c

BUKMUQ01_clean [0.051177   0.04801336] 4.263563204086627
BUKRUW01_clean [0.02250523 0.02605236] 3.455385192907016
BUKRUW02_clean [0.0205329 0.0250375] 3.2803433488216216
BUKRUW03_clean [0.01624613 0.02163385] 3.0038343499479194
BUKSIL_clean [0.02487585 0.0276276 ] 3.6015951660544046
BUKTAE_clean [0.02786591 0.02985324] 3.7337194151972013
BUKTEI_clean [0.02844656 0.03023935] 3.762853511065929
BUKTIM_clean [0.02825881 0.03016703] 3.7469793983753954
BUKTOS_clean [0.02885132 0.03056006] 3.7763437808625633
BULBER_clean [0.02806004 0.02149491] 5.221708568241173
BUNKUS_clean [0.02513953 0.03159512] 3.1827108793647745
BUNLAZ_clean [0.01925398 0.0318755 ] 2.416146987787697
BUPVIT_clean [0.02719781 0.02343074] 4.643098722113886
BUQWER_clean [0.24899662 0.08736688] 11.400046496412275
BURJOO_clean [0.12088894 0.04698154] 10.292462274300767
BURQIQ_clean [0.11810367 0.06358997] 7.429075344092683
BUSMUY_clean [0.03152533 0.02135902] 5.9038906154958735
BUSNAF_clean [0.04924608 0.0261683 ] 7.5275929785

cg401887b_si_004_clean [0.05200439 0.05002958] 4.157891051225817
cg500033b_si_002_clean [0.12425469 0.06597976] 7.532897551179976
cg500091k_si_002_clean [0.05126701 0.0422938 ] 4.8486546784726805
cg500149s_si_002_clean [0.06693509 0.02881415] 9.291975858927175
cg500175k_si_001_auto [0.02753526 0.03463265] 3.1802659491855843
cg500192d_si_003_clean [0.02295893 0.03225843] 2.846875794357918
cg501012e_si_002_clean [0.115013   0.06393511] 7.195608093434718
cg501141m_si_002_clean [0.17745024 0.08130916] 8.729656018140242
cg501231v_si_001_clean [0.05630227 0.03954701] 5.694718193616343
cg5012617_si_002_clean [0.0191184  0.01358472] 5.6293844593687
cg501279y_si_002_clean [0.11698395 0.06921534] 6.760579822845046
cg501332u_si_002_clean [0.03932576 0.0236613 ] 6.648116149303284
cg501560z_si_002_clean [0.00900921 0.01558051] 2.3129430263468227
cg501560z_si_003_clean [0.00709662 0.01439199] 1.9723809652757998
cg501730q_si_002_clean [0.04722097 0.03974637] 4.752229715487571
cg800424t_si_002_clean [

...

WULQUR_clean [0.07096242 0.055376  ] 5.125860675603949
WUNSEE_charged [0.28134654 0.09845622] 11.43032115549776
WUNSEE01_charged [0.19913078 0.07808282] 10.20100359553066
WUNSEE02_charged [0.24386092 0.08305918] 11.743959703433115
WUNSII_charged [0.31505095 0.09392456] 13.417191126822564
WUNSII01_charged [0.27774454 0.09561517] 11.6192661609654
WUNSII02_charged [0.24439551 0.09090054] 10.75441427119824
WUNTAB_charged [0.20934551 0.07390155] 11.331048912059359
WUNTEF_clean [0.1925363  0.08488786] 9.072501384386248
WUNTIJ_clean [0.1942331  0.08520391] 9.118506874149949
WUNTOP_clean [0.18106691 0.07917312] 9.147898066127716
WUNTUV_clean [0.18276381 0.08036234] 9.096988514806375
WUNVAD_clean [0.19079474 0.08167563] 9.34402291684861
WUNVEH_charged [0.22134146 0.0774873 ] 11.425948145231336
WUNVIL_charged [0.23006814 0.07924646] 11.612790684896009
WUNVOR_clean [0.19863726 0.08560302] 9.281787733107052
WUNVUX_clean [0.18449998 0.08178827] 9.023298267487863
WUPTUY01_clean [0.53837415 0.1172977

XEGJIE_clean [0.04972206 0.03550723] 5.601345233992711
XEGJIE02_clean [0.05012555 0.0355189 ] 5.6449447774712445
XEGJOK_clean [0.04043656 0.03330924] 4.855896609592086
XEGJUQ_clean [0.0449509  0.03477542] 5.170422333331908
XEGKAX_clean [0.03315963 0.0304236 ] 4.3597238372213045
XEGKEB_clean [0.03748982 0.03189973] 4.700956911083322
XEGKOL_clean [0.03526951 0.02845046] 4.958727172210164
XEGKUR_clean [0.03687157 0.03243029] 4.547793916010239
XEHDEV_clean [0.02941948 0.02871049] 4.09877849022909
XEKCAT_clean [0.12922636 0.08121845] 6.3643843675615805
XEKCAT01_clean [0.12117832 0.07799735] 6.214484058347913
XENCAU_clean [0.01120562 0.01901286] 2.3574822106562574
XENKIM_clean [0.05465364 0.04848551] 4.508864364336225
XEQGEH_clean [0.06585732 0.03747271] 7.02989702468888
XEQRAO_clean [0.0856291  0.03918371] 8.741294915284065
XESKAJ_clean [0.02859872 0.02487279] 4.599197463240403
XESKEN_SL [0.01987457 0.01900315] 4.183426847498335
XESKIR_clean_h [0.01014752 0.01196284] 3.39301322561856
XESKOX

YAPLUY_clean [0.03693166 0.02985671] 4.9478546342133
YAPMAF_clean [0.04651037 0.03476075] 5.3520555083224615
YAPRUE_charged [0.07093222 0.04356609] 6.51260818692403
YAQCEY_clean [0.06334239 0.04196151] 6.0381423866359265
YAQKEI_clean [0.1775718  0.05752097] 12.348318613000151
YARGAB_clean [0.04572976 0.03952569] 4.627851300049347
YARKEJ_clean [0.16949616 0.07885791] 8.597547674359664
YARKOT_clean [0.0818617  0.06803748] 4.812741972030405
YARYEV_clean [0.00530868 0.00763917] 2.7797143829878337
YARYIZ_clean [0.00524933 0.00754453] 2.783119059340145
YARYUL_clean [0.00599342 0.00990638] 2.4200245330416985
YARZEZ_clean [0.01510575 0.01753515] 3.4458217637692385
YARZEZ02_clean [0.00968657 0.0154186 ] 2.512955885565132
YARZEZ03_clean [0.01233507 0.01724586] 2.860992269621171
YASLOW_clean [0.02410434 0.02597703] 3.7116397484116623
YASLUA_clean [0.03759425 0.02533858] 5.934705286771963
YASLUC_clean [0.02409552 0.02565082] 3.757464853395169
YASMAJ_clean [0.02843311 0.02680196] 4.243437753291274


YOZQEK_clean [0.01469824 0.02220583] 2.64763529636465
YOZYOD_clean [0.13902616 0.06979619] 7.967550307150664
YOZYUJ_clean [0.07467071 0.05477572] 5.452833191978691
YUBFOR_clean [0.04700054 0.03134505] 5.997825886520694
YUBHAG_clean [0.04227922 0.0334366 ] 5.057837235700769
YUBQET_clean [0.24477943 0.09623468] 10.17427141773058
YUBTUM_clean [0.02516406 0.01870571] 5.381045715638901
YUDFEK_clean [0.05442434 0.03769395] 5.775392717777578
YUDFIO_clean [0.0185437  0.01989964] 3.727443955621345
YUDFOU_clean [0.01694364 0.01903556] 3.5604184524690976
YUFLIW_clean [0.09361044 0.06575292] 5.69467877873295
YUFLOC_clean [0.04439453 0.03837784] 4.62710074928278
YUGLES_clean [0.03364482 0.03703105] 3.634228539601906
YUGZIK_clean [0.02355965 0.03457865] 2.725341184670336
YUHHIU_clean [0.04638285 0.04179233] 4.43936508196276
YUJCOX_clean [0.1856777  0.08046724] 9.22997793722802
YUKBIP_clean [0.16785426 0.05922617] 11.336492167104568
YUKBIP01_clean [0.07886003 0.02973376] 10.608820710516332
YUKKAR_cle

ZUPBET_clean [0.12376904 0.05766191] 8.585843781545737
ZUQMAB_clean [0.10711513 0.05994854] 7.147137980649947
ZURROT_clean [0.02911656 0.02441134] 4.770989584269297
ZUWXUM_clean [0.09605803 0.07203469] 5.333987085277761
ZUXPOZ_clean [0.03254406 0.02095101] 6.213365011234875
ZUYHIM_clean [0.05220709 0.0480438 ] 4.34662494743787
ZUYLOW_clean [0.00741803 0.00734368] 4.040492695275305
ZUZZEB_clean [0.01463999 0.01955032] 2.9953451563594684


In [18]:
for M_list,h_list,dH,nam in zip(M_list_all.values,h_list_all.values,H_list_all.values,Name_list_all):
    q2,S2,= iso_mix([0.2,0.8], 298, M_list, h_list, dH)
    qH.append(q2)
    print(nam,q2,S2)

ABAYIO_clean [0.85174244 0.45195004] 7.5383768413137275
ABAYOU_clean [0.88123725 0.46961092] 7.506105305842239
ABEMIF_clean [0.83254142 0.50152942] 6.640020584949712
ABESUX_clean [0.17460904 0.12563743] 5.559140682387448
ABETAE_clean [0.17071019 0.1246589 ] 5.477673739656175
ABETIN_clean [0.29149409 0.36384444] 3.204601286760133
ABEXEM_clean [0.28248619 0.17826477] 6.338575876084914
ABEXEN_clean [0.53867759 0.44603538] 4.83080599379453
ABEXIQ_clean [0.35290678 0.20189778] 6.991791111144124
ABEXOW_clean [0.27921506 0.17511313] 6.377935740608752
ABEXUC_clean [0.27600316 0.17266756] 6.393862459203269
ABEYAJ_clean [0.28137954 0.16824017] 6.689948895625568
ABEYEN_clean [0.26950852 0.16080429] 6.704013230782518
ABEYIR_clean [0.26312783 0.15871078] 6.63163108841611
ABIXOZ_clean [0.07885749 0.09009647] 3.501024649387102
ABULOB_clean [2.46722595 0.78032955] 12.647097361267129
ABUWOJ_clean [0.64576908 0.46673998] 5.53429409557935
ac403674p_si_001_clean [0.13640622 0.13766237] 3.9635006382178726


ALUKOI_clean [0.1822451  0.27595878] 2.6416278646963107
ALUKUO_clean [0.18325949 0.27516415] 2.664002374847248
ALULAV_clean [0.1834247 0.2735583] 2.682056376655956
ALULEZ_clean [0.10593172 0.20241583] 2.0933485789075266
ALULID_clean [0.38844086 0.35100594] 4.426601560021234
AMAFOK_clean [0.97868225 0.35363815] 11.069871822833
AMAFUQ_clean [0.97660272 0.36768766] 10.624264309179328
AMAGAX_clean [0.96446443 0.39203396] 9.840621207012854
AMILOY_clean [1.37605707 0.48213133] 11.41645003349991
AMILUE_clean [1.4101554  0.49007296] 11.509758916431927
AMIMAL_clean [1.40411993 0.49143309] 11.428778031192941
AMIMEP_clean [1.39323185 0.49095052] 11.351301504754716
AMODUC_clean [0.1983134  0.29428819] 2.6954993474075732
AMOFAK_clean [0.20563136 0.29768527] 2.763070663899314
AMOYOR_clean [0.980184   0.37276403] 10.518010645434702
AMUFOF_clean [0.44345107 0.24684372] 7.185940557005359
AMUWIP_charged [0.31746288 0.33450135] 3.796252251272511
AMUWUB_clean [0.17056129 0.17996504] 3.7909871628401994
AMU

AXINAX01_clean [0.24686514 0.32527154] 3.0358036910244564
AXINIF_clean [0.61603793 0.4944768 ] 4.9833514750492025
AXIPEE_clean [0.274644   0.20326363] 5.404685445025825
AXIPEE01_clean [0.23064889 0.19973274] 4.61915043497813
AXIPEE02_clean [0.26239227 0.20514463] 5.116239628826394
AXIPEE03_clean [0.25745643 0.20269059] 5.080777071603004
AXUBAW_manual [0.22942735 0.27380303] 3.351713713115471
AXUBOL_clean [1.05015627 0.5244374 ] 8.009774112842335
AXUHAD_clean [0.42216432 0.35461835] 4.761900388335864
AXUHEH_clean [1.23922952 0.38776915] 12.783167839320344
AXUKUB_clean [0.28058712 0.28434391] 3.94715150183091
AXULAI_clean [0.39685118 0.30949668] 5.128987843954402
AXULEM_clean [0.33986789 0.29291985] 4.641104323783397
AYOQEM_clean [0.36561499 0.36615368] 3.9941151617262824
AYOXUJ_clean [0.1904055  0.14240434] 5.348305791702266
AYUJIP_clean [0.22947064 0.22616618] 4.0584429925115835
AZAFIS_clean [0.28698596 0.23027795] 4.985035937258029
AZAVOO_clean [1.85158933 0.65411521] 11.3227107220139

BOMCOX_clean [1.04976437 0.49442282] 8.492847241753894
BONMAT_clean [1.01801332 0.69648452] 5.846581147747406
BONMEX_clean [0.90617596 0.6680421 ] 5.425861362935294
BOPHIZ_clean [0.80791338 0.48711222] 6.634310082375529
BOTCEU_clean [0.49824549 0.18069711] 11.02940719598628
BOVHEB_clean [2.26360088 0.65297048] 13.866482122876741
BOWDIC_clean [1.22290702 0.56747244] 8.620027509188342
BOWNUY_clean [0.37449164 0.34070014] 4.396730083302342
BOWZUK_clean [0.41747593 0.33224241] 5.026160607733057
BOXFOK_clean [1.76299069 0.46058776] 15.310790474834699
BOYCEY_auto [0.62331092 0.48255023] 5.166806515265776
BOYTEQ_clean [1.1645439  0.32288405] 14.42677519389787
BOYYOF_clean [0.2737176  0.22011623] 4.974055702474696
BOZFIH_clean [1.51499049 0.68281691] 8.874944138215968
BOZFON_clean [1.11403483 0.40963904] 10.878209445149302
BUBJUF_clean [0.54748526 0.43100354] 5.081027947446528
BUCWAZ_clean [0.3474891  0.34360768] 4.0451843757808055
BUFNEW_charged [2.37744268 0.52706119] 18.043010761015804
BUFN

CAZNUO_charged [0.54668117 0.47006864] 4.651926307977306
CEBDOD_clean [0.44082297 0.18086816] 9.749045196399072
CECVAI_clean [0.23204798 0.26647938] 3.4831661456911127
CEDFIC_clean [0.75516088 0.43205395] 6.991357271400673
CEGDUO_clean [0.20792268 0.12228028] 6.801511687922569
CEHLAD_clean [0.38002198 0.36069401] 4.214342029772798
CEJBEA_clean [0.17255715 0.10967531] 6.293381550739382
CEJCAW_clean [0.4074102  0.27503873] 5.9251320486802985
CEJZAU_clean [1.02223606 0.5549181 ] 7.368554502982094
CEKHIL_clean [0.34340879 0.39825781] 3.449110373849599
CELZEA_clean [0.36590608 0.23198378] 6.309166452811504
CEMJOV_clean [1.31805558 0.58001386] 9.08982124723747
CENJIQ_clean [0.12055001 0.15466358] 3.1177349125752034
CENKAJ_clean [0.11528658 0.13781037] 3.346238135393307
CENPUI_clean [2.11803642 0.67838941] 12.488617262545002
CENQAP_clean [0.29417687 0.32877196] 3.579099304584385
CEQBUV_charged [0.09881339 0.05647257] 6.999036294375011
CETFOX_clean [0.28757574 0.20244631] 5.682014871831782
CET

...

WIZFES_clean [0.14840686 0.08746939] 6.786687272033539
WIZKOH_clean [0.86889453 0.62123899] 5.594591084786851
WIZMAV_clean [0.29377817 0.31572399] 3.7219619895417937
WIZMAV02_auto [0.1092548  0.16589774] 2.634268440657498
WIZMAV04_clean [0.29000476 0.31806943] 3.647062305627964
WIZMAV05_clean [0.29009197 0.3173913 ] 3.655953677901652
WIZMAV06_clean [0.29555145 0.32095388] 3.68341337916699
WIZMAV07_clean [0.29771004 0.32408242] 3.674497935962922
WIZMAV08_clean [0.29957869 0.32480101] 3.689381349169165
WIZMAV09_clean [0.30144144 0.3247439 ] 3.7129743818931367
WIZMAV10_clean [0.3014285  0.32550482] 3.7041356114487693
WIZMAV11_clean [0.29552275 0.32245463] 3.6659141883026014
WOBHEB_clean [0.34274655 0.31067512] 4.412925613441361
WOBHEB01_clean [0.30608969 0.28627371] 4.27688158430375
WOBHIF_clean [0.43693211 0.36696363] 4.762674762041934
WOBHIF01_clean [0.4274508  0.36241377] 4.717820779674492
WOBHIF02_clean [0.38088467 0.33806023] 4.506707832620648
WOBHIF03_clean [0.38281392 0.33783757] 4

XAKZIT_clean [0.54318661 0.3760199 ] 5.7782751091692655
XAKZOZ_clean [0.5714184  0.38437789] 5.9464231544567
XAKZUF_clean [0.5724126  0.38230522] 5.989063912944645
XALBAO_clean [0.48960314 0.36825704] 5.31805867710699
XALBES_clean [0.50249033 0.37455192] 5.366308914321629
XALBIW_clean [0.51412254 0.37199565] 5.5282639313825905
XALBOC_clean [0.50800446 0.37063926] 5.482467811726709
XALBUI_clean [0.53729637 0.37532094] 5.7262604939592965
XALCAP_clean [0.60435845 0.39561998] 6.110494698792244
XALCET_clean [0.58846454 0.38101338] 6.177888410367817
XALCOD_clean [0.69243008 0.37607187] 7.364869694899993
XALDAS_clean [0.49336933 0.43234842] 4.564553068049461
XALMII_clean [0.35084536 0.35181614] 3.988962571127977
XALMOO_clean [0.21140558 0.33362446] 2.5346532790644747
XALROT_clean [1.529109   0.71622066] 8.539876577806782
XALTIP_clean [0.17927034 0.26430273] 2.7131061651842923
XALXUF_clean [0.48368824 0.42050421] 4.601031143278367
XALXUF01_clean [0.50876581 0.43062797] 4.725803676742106
XAMDUM

XOJWOJ_clean [0.86049361 0.48267615] 7.131022394505111
XOKHIQ_clean [0.34242369 0.31139733] 4.398543705738372
XOKHOW_clean [0.53453468 0.39547638] 5.406489147493883
XOMCIN_clean [1.35992202 0.71908705] 7.56471426124401
XOMDOU_clean [1.44996012 0.45836283] 12.653383193130487
XOMGIQ_clean [0.39052978 0.28085372] 5.562038129798085
XONGUD_clean [0.02102966 0.02975439] 2.827099861429862
XONLES01_clean [0.19625427 0.33195896] 2.3648016338667164
XOPLOE_clean [0.58402723 0.42004734] 5.561537266812884
XOPMIA_clean [0.92657441 0.53072244] 6.98349523302052
XOPVII_clean [0.56769456 0.39328742] 5.773838925790772
XOQXIL_clean [0.70079542 0.40329773] 6.950650734290847
XORQAY_clean [0.93395737 0.32126583] 11.628468358586396
XOSPOK_clean [0.86015298 0.40231039] 8.552132960546784
XOTGUI_clean [0.19913015 0.1840712 ] 4.327241806633003
XOTHAP_clean [0.19770323 0.17684807] 4.47170796889169
XOTSEG_clean [0.54482833 0.40697627] 5.3548903764736195
XOTSIK_clean [0.51889799 0.37789929] 5.492447381631096
XOVPON_

YIRHIS_clean [0.40439779 0.21800958] 7.419816758297896
YISFEL_clean [0.11632704 0.08623629] 5.395734482866708
YIVZIO_clean [0.25724545 0.23029946] 4.468016613056041
YIWBEN_clean [0.68543184 0.51211812] 5.353701072248714
YIWLEV_clean [0.08774555 0.10917927] 3.2147330999662285
YIWLIZ_clean [0.09705778 0.11422859] 3.398721165860849
YIWLOF_clean [0.09304398 0.11196438] 3.3240563611865213
YIWMOG_clean [0.0929271  0.10883795] 3.4152463389156082
YIWMUM_clean [0.10012767 0.11255445] 3.5583729731950036
YIWNAT_clean [0.09702429 0.11097617] 3.4971216565524315
YIWNIB_clean [0.08513771 0.10741886] 3.1703077921430443
YIWNOH_clean [0.09090568 0.11100166] 3.2758315197749655
YIWPAV_clean [0.08765357 0.10868466] 3.2259775108284856
YIWPEZ_SL_part [0.08982614 0.10958369] 3.2788142723346216
YIWPID_clean [0.09227905 0.1093781 ] 3.374681091904476
YIWSEE_clean [0.34194122 0.25536641] 5.3560876648189
YIXYUB_clean [0.42695349 0.25396409] 6.724627734725747
YIXZAI_clean [0.43345515 0.25443471] 6.814402848265721
Y

ZAVRIZ_clean [0.5299873  0.32827892] 6.457768328034099
ZAZBUZ_clean [0.82264734 0.47451978] 6.93456730455292
ZECKID_clean [0.13337985 0.2120641 ] 2.51584026458388
ZECKOJ_clean [0.14915733 0.22477978] 2.654283694001407
ZEDQEG_clean [0.41541815 0.30247246] 5.493632711527638
ZEDVUB_charged [0.24024362 0.15255413] 6.299235999638873
ZEDWAI_charged [0.24983054 0.15606511] 6.4032387099504415
ZEDZAL_clean [1.04015358 0.59970138] 6.937810109363318
ZEDZAL01_clean [1.03939583 0.59977912] 6.931857435255649
ZEKRIS_clean [1.87010692 0.70710758] 10.578910329289094
ZELQAK_clean [0.55400958 0.49106828] 4.512688858977692
ZELROZ_manual [0.14648337 0.26730303] 2.192019560866539
ZESTIC01_clean [1.78482236 0.57762461] 12.359739003413624
ZESZEE_clean [1.39139054 0.65576593] 8.487117019002193
ZETLER_clean [0.83062501 0.4411747 ] 7.531030366581357
ZEVWUU_clean [0.36724293 0.35399982] 4.149639657872418
ZEVXAB_clean [0.48034931 0.41115588] 4.673160010155645
ZEVXEF_clean [0.27328247 0.26522297] 4.121550529857905


In [20]:
for i in range(6367):
    WC1 = qH[i][0] - qL[i][0] 
    WC2 = qH[i][1] - qL[i][1] 
    print(Name_list_all[i],WC1,WC2)

ABAYIO_clean 0.7515460046638494 0.398704378920357
ABAYOU_clean 0.7768275840859635 0.4138371889383747
ABEMIF_clean 0.7320981204689273 0.44116783320421704
ABESUX_clean 0.15491330755068686 0.11144474393099535
ABETAE_clean 0.1515013296701077 0.11061863970943098
ABETIN_clean 0.26140253372578903 0.32629815260428685
ABEXEM_clean 0.2483031029745054 0.156699395515003
ABEXEN_clean 0.48165261715871743 0.39839702839454505
ABEXIQ_clean 0.308430778842085 0.17648958804313197
ABEXOW_clean 0.2457652560140055 0.15435239965475403
ABEXUC_clean 0.24242889431660927 0.15180357139978343
ABEYAJ_clean 0.24755657874067952 0.14798886054718488
ABEYEN_clean 0.23654614242094082 0.14114315394251067
ABEYIR_clean 0.23060321451783303 0.13915016183967108
ABIXOZ_clean 0.07069002764299923 0.08077273498678318
ABULOB_clean 1.9346943666996046 0.6075857042803754
ABUWOJ_clean 0.5691855516452756 0.4112832331389205
ac403674p_si_001_clean 0.12221069367269805 0.12332834999068502
ACAJIZ_clean 0.3874912516327796 0.2177036982170202
AC

EFOSUO_clean 0.04427103546445252 0.059094459037055444
EFOTAV_clean 0.042791870931715936 0.0583335675999713
EFOTEZ_clean 0.02157550178324251 0.03364598244591798
EFOTID_clean 0.022187992057784676 0.033002974142086784
EGATAI_clean 0.31737069042901234 0.299132753664806
EGATEM_clean 0.9823371117663102 0.48793671587667137
EGATIQ_clean 0.9792124002169911 0.48775806795747456
EGATOW_clean 0.6323383970715734 0.4175293810738487
EGEHOO_charged 0.6086054554138558 0.2945644785463872
EGEJIK_manual 0.2824090045231109 0.30206548853371923
EGELUY_SL 1.110093702778612 0.6411029351289276
EGELUY01_manual 1.0904203061805489 0.6366904733790917
EGIFUV_clean 1.845367243223788 0.3798879600651137
EGOZAB_clean 0.6034483208033858 0.385350711039312
EHALOP_manual 1.066657651442049 0.6411934495227657
EHETER02_clean 0.7171205096890391 0.3085533291177295
EHIJAH_clean 0.17173497484977063 0.2779077608425015
EHODOV_SL 0.8348578620155443 0.4408768706675775
EJAXIW_clean 0.4174170765229213 0.3673150300281636
EJETAQ_clean 0.11

IQOKIK_clean 0.6138521101558609 0.41324686639562286
IQUCOO_clean 1.0719946821821147 0.49969373027703823
IRAGEN_charged 0.305469861106072 0.17750759711572678
IROVAO_clean 0.17742684420713867 0.12494321968698696
IROVES_clean 0.15592111796486594 0.09803257503304444
ISENUQ_clean 0.5190314838133242 0.285077240105191
ISOHEE_clean 0.43937807386549 0.3737595637320806
ISOJOQ_clean 0.5494497242617093 0.3659755025469974
ITAHEQ_clean 0.1088916616827999 0.06048706882893337
ITAPAU_clean 0.02767823130077802 0.055978051874265605
ITAPIC_clean 0.037507966301307226 0.07411691336885211
ITIPOS_clean 0.1711137919411344 0.15503782296778673
ITIWAK_clean 0.9709053313580035 0.424761260774061
ITIWEO_clean 1.0409272615161478 0.4761616855895611
ITIWIS_charged 1.0974831840149402 0.3581021719506591
ITIWOY_charged 1.1228268250796551 0.3585026367656152
ITIWUE_charged 1.1102954463645371 0.3613668295655985
ITOSEQ_clean 1.203934795553083 0.5569506028047948
ITUVID_clean 1.422525617028074 0.7336708476756265
ITUVOJ_clean 0.

NERVEN_clean 0.2621587215527319 0.19776852428029015
NESSOV_clean 1.2433412703492903 0.5019609084110427
NESTAI_clean 0.8582133479912687 0.5171360044804499
NESVAK_charged 0.29521908709356465 0.2672001676992672
NESVEO_charged 0.512486625731768 0.4284157321511765
NETHOJ_clean_h 0.2732292886169954 0.1941541893313452
NETYIV_clean 0.23923057787820798 0.26636339038132995
NETYIV01_clean 0.25891491055883015 0.27976214527346394
NEVPOU_clean 0.40727311551623374 0.263941365585461
NEVPUA_charged 0.43076514378945807 0.2272216340739287
NEVQAH_charged 0.4204120299925599 0.21043470766496888
NEVVAM_clean 0.5997080350990203 0.33119073667065446
NEVVAM01_clean 0.5764248667735111 0.3224518415794452
NEVVAM02_clean 0.5718846273030117 0.31866528781891307
NEVVIU_clean 0.5752551773299691 0.30522718522007464
NEVVIU01_clean 0.5932272269843561 0.3109988162532596
NEVVIU02_clean 0.5771768164678551 0.30844044756056055
NEXVET_clean 0.10911371353636522 0.19036167301524257
NEXVUI_clean 0.07070527054711703 0.08163879289931

SEMNEF_clean 0.24002194562679155 0.3581946380864826
SEMNIJ_clean 0.22462457293406446 0.36548009295118555
SEQSIS_clean 1.3534785869367467 0.4933502527099181
SEQSOY_clean 1.1449476349387568 0.5877227639288215
SEQTEP_clean 0.8311725242921294 0.443373673822998
SERCEZ_clean 0.36913056649474923 0.37491737444771256
SERJOP_clean 0.2161608540720875 0.17134888761620184
SERJUV_clean 0.21597553423102006 0.1875025193330135
SERKAC_clean 0.12538914365462678 0.11994165013218981
SERKOQ_clean 0.38268008656946095 0.3353317714470221
SERLEI_clean 0.5779706920709282 0.45257108534909973
SERWAP_clean 1.207342903164051 0.645329155406721
SETDUS_clean 0.3995004216211445 0.3283728334541536
SETQAL_clean 1.248503510594206 0.48793788859239345
SETQEP_clean 1.2488362830116266 0.48653054549737096
SETQIT_clean 1.2341695336103131 0.48830960570753845
SETQOZ_SL 0.5015770923404991 0.392484403374964
SETQUF_SL 0.5014486959109155 0.39222565291201517
SETRAM_clean 0.5496518810713528 0.42777279133131185
SETSIV_clean 0.39695107273

XUKYEI_neutral 0.1125097154682044 0.2444775247292204
XUMMOJ_clean 0.22155644656997064 0.1893519121859241
XUMMUP_clean 0.17031697716235986 0.16111354706757924
XUMNIE_clean 0.7993943843084688 0.5579442784421097
XUMTUV_clean 0.8309056519524642 0.4303020503089635
XUMVAD_clean 0.6768782268771506 0.3861551570473777
XUMVEH_clean 0.5951547335660411 0.3634433737971828
XUNGUJ_clean 1.2435923703672895 0.5716276681094858
XUNHAQ_clean 1.0707825180569246 0.5776342298498385
XUNJEW_clean 1.5100492127088327 0.4808401685448709
XUNSEG_clean 1.2324784963636453 0.46016821875158487
XUNSOQ_clean 1.3971805975311444 0.4693336394764184
XUPGIA_clean 0.14186795058566304 0.2407401054638003
XUPSAE_clean 1.5672093672389045 0.7591798492648762
XUTQEI_clean 0.22286132031522743 0.2988022950485791
XUVHEB_clean 1.3725507478048304 0.5126463574530405
XUVPAH_clean 0.3264783592839778 0.2608831617054903
XUVQOW_clean 0.6592678293574742 0.42034250022163827
XUWFUR_clean 0.5422902115145545 0.322667831614324
XUWKEH_clean 0.08192837